In [ ]:
import librosa  # https://librosa.org/
import librosa.display

# import librosa.beat
import sounddevice as sd  # https://anaconda.org/conda-forge/python-sounddevice
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy.stats as scs
from scipy.spatial.distance import (
    euclidean,
    cosine,
    cityblock, # cityblock -> Manhattan
) 

from librosa.feature import (
    mfcc,
    spectral_centroid,
    spectral_bandwidth,
    spectral_contrast,
    spectral_flatness,
    spectral_rolloff,
    zero_crossing_rate,
)
from librosa import yin
from librosa.beat import tempo


In [ ]:
fName = "Queries/MT0000202045.mp3"
sr = 22050
mono = True
warnings.filterwarnings("ignore")


In [ ]:
y, fs = librosa.load(fName, sr=sr, mono = mono)
print(y.shape)
print(fs)

In [ ]:
sd.play(y, sr, blocking=False)

In [ ]:
plt.figure()
librosa.display.waveshow(y)

In [ ]:
Y = np.abs(librosa.stft(y))
Ydb = librosa.amplitude_to_db(Y, ref=np.max)
fig, ax = plt.subplots()
img = librosa.display.specshow(Ydb, y_axis='linear', x_axis='time', ax=ax)
ax.set_title('Power spectrogram')
fig.colorbar(img, ax=ax, format="%+2.0f dB")

In [ ]:
rms = librosa.feature.rms(y = y)
rms = rms[0, :]
print(rms.shape)
times = librosa.times_like(rms)
plt.figure(), plt.plot(times, rms)
plt.xlabel('Time (s)')
plt.title('RMS')

## Semana 2

In [ ]:
features = pd.read_csv("dataset/Features - Audio MER/features.csv", delimiter=';')
features

In [ ]:
# Read top100_features.csv file and remove first row and col and last col
top_100 = pd.read_csv("dataset/Features - Audio MER/top100_features.csv", header=None).iloc[1:, 1:-1]
top_100

In [ ]:
top_100_norm = np.apply_along_axis(librosa.util.normalize, 1, top_100.to_numpy(dtype = np.float32))
np.savetxt("top_100_features_normalized.csv", top_100_norm, delimiter=";")

top_100_norm.shape

In [ ]:
#get columns that represent the value of a feature
#filtered_columns = list(filter(lambda col: col[0]=="F", top_100.columns))
#normalize those columns values with the librosa library
#top_100[filtered_columns] = librosa.util.normalize(top_100[filtered_columns].to_numpy())

In [ ]:
# top_100.to_csv("top_100_extracted_features.csv", sep = ";", index = False)

In [ ]:
#Receives uniform signal and extracts features
def extract_features(signal : np.array):
    mean = signal.mean()
    stdDev = signal.std()
    skewness = scs.skew(signal)
    kurtosis = scs.kurtosis(signal)
    median = np.median(signal)
    max_value = signal.max()
    min_value = signal.min()
    return (mean, stdDev, skewness, kurtosis, median, max_value, min_value)

In [ ]:
# Código para extrair os features de todas as músicas em main.py

In [ ]:
path = "dataset/MER_audio_taffc_dataset/Q1/MT0000040632.mp3"
y,fs = y, fs = librosa.load(path, sr=sr, mono = mono)

In [ ]:
# Spectral features extraction


# Black formmating :)
def get_feature_vector(filename):
    y, fs = librosa.load(filename, sr=sr, mono=mono)

    _mfcc = np.apply_along_axis(extract_features, 1, mfcc(y=y, n_mfcc=13)).flatten()

    spc_centroid = np.apply_along_axis(
        extract_features, 1, spectral_centroid(y=y)
    ).flatten()

    spc_bdwth = np.apply_along_axis(
        extract_features, 1, spectral_bandwidth(y=y)
    ).flatten()

    spc_contrast = np.apply_along_axis(
        extract_features, 1, spectral_contrast(y=y)
    ).flatten()

    spc_flatness = np.apply_along_axis(
        extract_features, 1, spectral_flatness(y=y)
    ).flatten()

    spc_rollof = np.apply_along_axis(
        extract_features, 1, spectral_rolloff(y=y)
    ).flatten()

    f0 = np.apply_along_axis(extract_features, 0, yin(y, 20, 11025))
    f0[f0 == 11025] = 0

    rms = np.apply_along_axis(extract_features, 1, rms(y=y)).flatten()

    zcr = np.apply_along_axis(extract_features, 1, zero_crossing_rate(y=y)).flatten()

    _tempo = tempo(y=y)

    feature_vector = np.concatenate(
        (
            _mfcc,
            spc_centroid,
            spc_bdwth,
            spc_contrast,
            spc_flatness,
            spc_rollof,
            f0,
            rms,
            zcr,
            _tempo,
        )
    )

    return feature_vector


In [ ]:
# feature_vector.tofile("exercise2_features.csv", sep = ";")

In [ ]:
librosa.feature.zero_crossing_rate(y=y)

In [ ]:
librosa.feature.spectral_contrast(y=y).shape

## Semana 3

In [ ]:
normalized_features = pd.read_csv("exercise2_features_normalized.csv", header=None, sep=';')

In [ ]:
normalized_features

In [ ]:
features = normalized_features.to_numpy()

In [ ]:
def similarity_metric(x, y, metric = 'euclidean'):
    # if x.shape != y.shape:
    #     return
    
    if metric == 'euclidean':
        return euclidean(x, y)
    elif metric == 'cosine':
        return cosine(x, y)
    elif metric == 'manhattan':
        cityblock(x, y)

In [ ]:
x = get_feature_vector('Queries/MT0000202045.mp3')

results = []
temp = []

for i in features:
    for j in features:
        temp.append(similarity_metric(i, j, 'cosine'))
    results.append(temp)

results = np.array(results)
results.shape
results